NAATOS Bulk Analysis Tool for Modules
============

Simon Ghionea, Started 3/17/2025

This notebook focuses on data downloaded during naatos v1 stage gate 4 4/7/2025

## Imports


In [13]:
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import math
import time

import plotly.express as px
import panel as pn
import param

import naatos_module_tools.logreader as logreader
import naatos_module_tools.logprocessors as logprocessors
import naatos_module_tools.logplotter as logplotter

In [14]:
pn.extension("plotly")
pn.extension("tabulator")

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Enumerate, Select Experiment Folders, Load All Data

In [85]:
root = r'C:\Users\SimonGhionea\Global Health Labs, Inc\NAATOS Product Feasibility - General - Internal - Electronic Control Module\Beta design\PowermoduleTestData\by_exp'
#root = r'C:\Temp\NAATOS_MODULE_AUTODOWNLOADS'
rootpath = Path(root)

experiment_list = [x.name for x in rootpath.iterdir() if x.is_dir()];

experiments_to_plot = [
    #'20250128_sgdev_3.1',
    #'20250128_sgdev_3.1_b_abridge_pwmbugsearch',
    #'20250128_sgdev_3.1_c_abridge_diffpid',
    #'20250128_sgdev_3.1_d_2cycle_emulate',
    
    #'20250225_sgdev_cooldown_test',
    #'20250228_sgdev_cooldown_test',

    #'20250311_DX_download',
    '20250404_PM_fromDX',
    
    # '20250310_SimonGhionea_autodl',
    # '20250311_SimonGhionea_autodl',
    #'20250312_SimonGhionea_autodl',
    #'20250313_SimonGhionea_autodl_PM_eelab_continuous',
    #'20250317_SimonGhionea_autodl',
]

experiments_to_plot = [x for x in experiments_to_plot if x in experiment_list]

print('Found experiment folders:',experiments_to_plot)

Found experiment folders: ['20250404_PM_fromDX']


In [86]:
#%% Load associated datafile from the unit-logged run
# logfile = logfilenames[0];
# df_in,df_events = logreader.scanALogfile(logfile)
dfraw = logreader.processRootFolder(rootpath,experiments_to_plot);
df_events = dfraw[ ~dfraw['Event'].isnull() & ~(dfraw['Event']==' ') ]

Processing rootfolder C:\Users\SimonGhionea\Global Health Labs, Inc\NAATOS Product Feasibility - General - Internal - Electronic Control Module\Beta design\PowermoduleTestData\by_exp
Experiment 20250404_PM_fromDX
The folder we will assume each folder are UNITS
folder "Unit 16"
['config', 'logs']
Folder logs
Done loading log sample_03-14-25_113603.csv
Done loading log sample_03-20-25_122020.csv
Done loading log sample_03-25-25_102548.csv
Done loading log sample_03-25-25_111128.csv
Done loading log sample_03-25-25_111136.csv
Done loading log sample_03-25-25_144404.csv
Done loading log sample_03-25-25_154952.csv
Done loading log sample_03-26-25_130431.csv
Done loading log sample_03-26-25_145144.csv
Done loading log sample_03-27-25_120750.csv
Done loading log sample_03-28-25_105940.csv
Done loading log sample_03-28-25_142443.csv
Done loading log sample_03-28-25_153146.csv
Done loading log sample_03-28-25_162715.csv
Done loading log sample_03-31-25_142824.csv
Done loading log sample_03-31-2

# Pull Out Relevant Information

In [87]:
#%% Filter
df = dfraw;

#df = df[df['expname']=='20250311_SimonGhionea_autodl']
#df = df[df['unit']=='PM11'];
#df = df[df['run']=='sample_03-11-25_153808']
#df = df[df['run']=='sample_03-11-25_153400']

In [88]:
#%% Pull out by-cycle information

#dfbuildlist = [];
def convert_dtypes(df : pd.DataFrame):
    # try to convert strings to numerics as appropriate
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col]);
        except:
            pass
    
    return df

buildlist = [];
for unit,dfunits in df.groupby('unit'):
    #unit_run_counter = 0;
    for expname,dfexps in dfunits.groupby('expname'):
        for run,dfrun in dfexps.groupby('run'):
            #print('unit:{:s} exp:{:} run:{:s} shape:{:s}'.format(unit,expname,run,str(dfrun.shape)))
            
            dfrunevents = dfrun[ ~dfrun['Event'].isnull() & ~(dfrun['Event']==' ') ]
            dfrunevents = dfrunevents[['Time','Event']]
            
            status = 'nostatus';
            text = '';

            # Cycle Analysis
            evts_beg = dfrunevents[dfrunevents['Event'].str.match(r'^Cycle \d+ Started')]
            evts_end = dfrunevents[dfrunevents['Event'].str.match(r'^Cycle \d+ Stopped')]
            evts_nocyc = dfrunevents.loc[dfrunevents.index.symmetric_difference(evts_beg.index.tolist()+evts_end.index.tolist())]
            
            if( evts_beg.shape[0]==0 ):
                # was not a run
                #text+='Not A Run: {:s}'.format(str('\n'.join(dfrunevents['Event'].to_list())));
                text+='Not A Run: {:s}'.format(str(dfrunevents['Event'].to_list()));
                status = 'notrun';
                if(any(dfrunevents['Event'].str.startswith('Boot'))):
                    status = 'bootup';
                    
                    # check for boot reset reasons
                    reset_reason = dict( [tuple(s.split('=')) for s in dfrunevents['Event'].str.extract(r'POWER.RESETREAS=[^\s]*\s(.*)').dropna().iloc[0].item().split(' ')] );
                    # all reasons 0, I think that means power-on (turned on from switch) based on my reading of Nordic datasheet
                    if(all([v=='0' for k,v in reset_reason.items()])):
                        status+=',POR';
                
                dfrunevents['expname'] = expname;
                dfrunevents['unit'] = unit;
                dfrunevents['run'] = run;
                dfrunevents['status'] = status;
                dfrunevents['text'] = text;
                dfrunevents['Cycle'] = 0;
                dfrunevents = dfrunevents.rename(columns={'Time':'TimeBeg'})
                #buildlist.append(dfrunevents);
                buildlist.append( dfrunevents.drop(columns=['Event']).iloc[[0]] );
            
            elif( evts_beg.shape[0]==evts_end.shape[0]):
                #print('here5')
                # all Start and Stopped are matched
                ncycles = evts_beg.shape[0];
                cyc_num_beg = evts_beg['Event'].str.extract(r'Cycle (\d+) ',expand=False).astype(np.uint8);
                cyc_num_beg.name='Cycle'
                cyc_num_end = evts_end['Event'].str.extract(r'Cycle (\d+) ',expand=False).astype(np.uint8);
                cyc_num_end.name='Cycle'
                earlyterm = any(evts_end['Event'].str.match(r'.*early\.'));
                if(cyc_num_beg.shape[0]>0):
                    fn_lmbda_split_keyvalue_strings = lambda x: dict(tuple([tuple(z.split('=')) for z in x]));

                    #cyc_data_beg = evts_beg['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ').apply(lambda x: x[1:]).apply(lambda x: dict(tuple([tuple(z.split('=')) for z in x]))).apply(pd.Series)
                    #cyc_data_end = evts_end['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ').apply(lambda x: x[1:]).apply(lambda x: dict(tuple([tuple(z.split('=')) for z in x]))).apply(pd.Series)
                    cyc_data_beg = evts_beg['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ');
                    if(not all(cyc_data_beg.isna())):
                        # there are fields after "Cycle N started." messages
                        cyc_data_beg = cyc_data_beg.apply(lambda x: x[1:]).apply(fn_lmbda_split_keyvalue_strings).apply(pd.Series)
                        cyc_data_beg = convert_dtypes(cyc_data_beg);

                        evts_beg = pd.concat((evts_beg,cyc_num_beg,cyc_data_beg),axis='columns')
                        #print('here1');
                    else:
                        evts_beg = pd.concat((evts_beg,cyc_num_beg),axis='columns');
                        #print('here2');
                    evts_beg = evts_beg.rename(columns={'Time':'TimeBeg'})


                    cyc_data_end = evts_end['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ');
                    if(not all(cyc_data_end.isna())):
                        # there are fields after "Cycle N started." messages
                        cyc_data_end = cyc_data_end.apply(lambda x: x[1:]).apply(fn_lmbda_split_keyvalue_strings).apply(pd.Series)
                        cyc_data_end = convert_dtypes(cyc_data_end);

                        evts_end = pd.concat((evts_end,cyc_num_end,cyc_data_end),axis='columns')
                        #print('here3')
                    else:
                        evts_end = pd.concat((evts_end,cyc_num_end),axis='columns');
                        #print('here4');
                    evts_end = evts_end.rename(columns={'Time':'TimeEnd'})

                    # Combine Beginning and Ending of cycle information into a single cycle row
                    cyc_data = pd.merge(evts_beg,evts_end,on='Cycle',suffixes=('Beg','End')).set_index('Cycle',drop=True);

                    # calculate cycle information
                    cyc_data['CCycleRTCSeconds'] = (cyc_data['TimeEnd']-cyc_data['TimeBeg']).apply(lambda x: x.total_seconds());
                    #cyc_data.loc[0,['EventEnd']] = 'norm'
                else:
                    print('\tOTHER')

                
                if(earlyterm):                   
                    # print('\tEarly abort in cycle {:d} @ {:.0f} s @ total runtime {:.0f} s:'.format(
                    #     cyc_data.index[-1],
                    #     cyc_data['CalcCycleRTCSeconds'].iloc[-1].item(),
                    #     cyc_data['CalcCycleRTCSeconds'].sum().item()
                    #     )
                    # )
                    text += 'Early abort in cycle {:d} @ {:.0f} s @ total runtime {:.0f} s:\n'.format(
                        cyc_data.index[-1],
                        cyc_data['CCycleRTCSeconds'].iloc[-1].item(),
                        cyc_data['CCycleRTCSeconds'].sum().item()
                    );
                    #print('\t',str(evts_nocyc['Event'].to_list()))
                    #text += '{:s}'.format('\n'.join(evts_nocyc['Event'].to_list()));
                    text += '{:s}'.format(str(evts_nocyc['Event'].to_list()));
                    normal_interruptions = [
                        'HALL sensor interrupted',
                        'Optical sensor interrupted',
                        'ButtonCycle cancled via button click',
                    ]
                    if(any(pd.concat([evts_nocyc['Event'].str.contains(s) for s in normal_interruptions]))):
                        status = 'run_ended_early_user';
                    else:
                        status = 'run_ended_early_other';
                    # if(evts_nocyc['Event'].str.contains('Optical sensor interrupted')|evts_nocyc['Event'].str.contains('Hall sensor interrupted')):
                    #     status = 'run_ended_early';
                else:
                    # print('\tCompletedRun @ total runtime {:.0f} s'.format(
                    #     cyc_data['CalcCycleRTCSeconds'].sum().item()
                    # ))
                    text += 'CompletedRun @ total runtime {:.0f}s:'.format(
                        cyc_data['CCycleRTCSeconds'].sum().item()
                    )
                    #text += '{:s}'.format('\n'.join(evts_nocyc['Event'].to_list()));
                    text += '{:s}'.format(str(evts_nocyc['Event'].to_list()));
                    status = 'run_success';
                
                try:
                    if(not earlyterm):
                        if( not all(cyc_data_beg['runtime_s']==cyc_data_end['expected_sec']) ):
                            #print('\tMismatched runtimes');
                            text += '\n\tMismatched runtimes';
                            status = 'run_mismatched_runtimes';
                    else:
                        if( not all(cyc_data_beg['runtime_s'][0:-1]==cyc_data_end['expected_sec'][0:-1]) ):
                            #print('\tMismatched runtimes (early)');
                            text += '\n\tMismatched runtimes (early)';
                            status = 'run_mismatched_runtimes_early';
                except:
                    pass;
                
                # SUMMARIZE PER RUN
                if(not earlyterm):
                    pass;
                    # for (k1,v1),(k2,v2) in zip(evts_beg.iterrows(),evts_end.iterrows()):
                    #     #print( 'Cycle{:d} idx{:d} to idx{:d}'.format(v1['Cycle'],k1,k2) );
                        
                    #     # subset this cycle in the run
                    #     dfruncyc = dfrun.loc[k1:k2]

                    #     # last third
                    #     dfruncyclst3rd = dfruncyc.loc[k2-((k2-k1)//3):]

                    #     # calculate some summaries
                    #     #cyc_data.loc[v1['Cycle'],'C'] = cyc_data['CCycleRTCSeconds'].sum().item()
                    #     cyc_data.loc[v1['Cycle'],'CL3MedAmpTemp'] = dfruncyclst3rd['AmpTemp'].median()
                    #     cyc_data.loc[v1['Cycle'],'CL3MedValveTemp'] = dfruncyclst3rd['ValveTemp'].median()
                    #     cyc_data.loc[v1['Cycle'],'CL3MaxAmpPWM'] = dfruncyclst3rd['AmpPWM'].max()
                    #     cyc_data.loc[v1['Cycle'],'CL3MaxValvePWM'] = dfruncyclst3rd['ValvePWM'].max()
                    #     cyc_data.loc[v1['Cycle'],'CMaxAmpTemp'] = dfruncyc['AmpTemp'].max()
                    #     cyc_data.loc[v1['Cycle'],'CMaxValveTemp'] = dfruncyc['ValveTemp'].max()

                    #     pass
                for (k1,v1),(k2,v2) in zip(evts_beg.iterrows(),evts_end.iterrows()):
                    #print( 'Cycle{:d} idx{:d} to idx{:d}'.format(v1['Cycle'],k1,k2) );
                    
                    # subset this cycle in the run
                    dfruncyc = dfrun.loc[k1:k2]

                    # last third
                    dfruncyclst3rd = dfruncyc.loc[k2-((k2-k1)//3):]

                    # calculate some summaries
                    #cyc_data.loc[v1['Cycle'],'C'] = cyc_data['CCycleRTCSeconds'].sum().item()
                    #cyc_data.loc[v1['Cycle'],'C'] = cyc_data['CCycleRTCSeconds'].sum().item()
                    cyc_data.loc[v1['Cycle'],'CL3MedAmpTemp'] = dfruncyclst3rd['AmpTemp'].median()
                    cyc_data.loc[v1['Cycle'],'CL3MedValveTemp'] = dfruncyclst3rd['ValveTemp'].median()
                    cyc_data.loc[v1['Cycle'],'CL3MaxAmpPWM'] = dfruncyclst3rd['AmpPWM'].max()
                    cyc_data.loc[v1['Cycle'],'CL3MaxValvePWM'] = dfruncyclst3rd['ValvePWM'].max()
                    cyc_data.loc[v1['Cycle'],'CMaxAmpTemp'] = dfruncyc['AmpTemp'].max()
                    cyc_data.loc[v1['Cycle'],'CMaxValveTemp'] = dfruncyc['ValveTemp'].max()

                    pass
                #cyc_data['CRTCRuntime'] = (dfrun['Time'].iloc[[0,-1]]).diff().apply(lambda x: x.total_seconds()).iloc[-1].item();
                cyc_data['CRTCRuntime'] = ( cyc_data.iloc[-1]['TimeEnd'] - cyc_data.iloc[0]['TimeBeg'] ).total_seconds();

                cyc_data['expname'] = expname;
                cyc_data['unit'] = unit;
                cyc_data['run'] = run;
                cyc_data['status'] = status;
                cyc_data['text'] = text;
                buildlist.append(cyc_data.reset_index());
            else:
                text += 'Run cycles non-sensical, nbegun={:d} nended={:d}'.format(evts_beg.shape[0],evts_end.shape[0]);
                status = 'nonsensical_cycles';

                dfrunevents['expname'] = expname;
                dfrunevents['unit'] = unit;
                dfrunevents['run'] = run;
                dfrunevents['status'] = status;
                dfrunevents['text'] = text;
                dfrunevents['Cycle'] = 0;
                dfrunevents.rename(columns={'Time':'TimeBeg'});
                buildlist.append(dfrunevents);

            if(status == 'notrun'):
                print('unit:{:s} exp:{:} run:{:s} shape:{:s} status:{:s}'.format(unit,expname,run,str(dfrun.shape),status))
                print(text)
            #break;


#dflifetest = pd.concat(dfbuildlist)
dfbuilt = pd.concat(buildlist,ignore_index=True)
dfbuilt['Cycle'] = dfbuilt['Cycle'].astype(np.uint8)
dfbuilt = dfbuilt.set_index(['unit','expname','run','Cycle'])
#dfbuilt = pd.concat(buildlist).reset_index()

#%% Show run summary

In [89]:
#%% Pull out only whole-run-summaries
#%% Show run summary
#for (unit,expname,run),dfgrp in dfbuilt.xs(1,level='Cycle').groupby(['unit','expname','run']):
runbuild = [];
for (unit,expname,run),dfgrp in dfbuilt.groupby(['unit','expname','run']):
    dfcyclesorted = dfgrp.sort_values('Cycle');
    dffirst = dfcyclesorted.reset_index().iloc[0];
    dflast = dfcyclesorted.reset_index().iloc[-1];
    #print(unit,expname,run,dffirst.shape)
    #print(dffirst['text'])

    calc_cols_from_cycles_to_sum = ['I2CERRCOUNT'];
    calc_cols_from_cycles_to_max = ['tick_delta_sec','rtc_delta_sec'];
    calc_cols_from_cycles_to_first = ['CRTCRuntime'];

    cols_to_always_include = ['status','text']

    runseriesitem = dfgrp.reset_index(level='Cycle').iloc[0][cols_to_always_include+calc_cols_from_cycles_to_first];
    runseriesitem['TimeBeg'] = dffirst['TimeBeg'];
    runseriesitem['TimeEnd'] = dflast['TimeEnd'];
    
    # items to max across cycles
    for col in calc_cols_from_cycles_to_max:
        if(col in dfgrp.columns):
            runseriesitem['cycmax_'+col] = dfgrp[col].max();
    
    # items to sum across cycles
    for col in calc_cols_from_cycles_to_sum:
        if(col in dfgrp.columns):
            runseriesitem['cycsum_'+col] = dfgrp[col].sum();
    
    # get other events
    if(runseriesitem['status']=='run_success'):
        dfrunevents = df[df['Event']!=' '].query("unit==@unit and expname==@expname and run==@run")
        
        fn_lmbda_split_keyvalue_strings = lambda x: dict(tuple([tuple(z.split('=')) for z in x]));
        
        # get parameters after "Run complete"
        inlineparams = dfrunevents[dfrunevents['Event'].str.startswith('Run complete')]['Event'].str.extract(r'Run complete. (.*)',expand=False);
        if(inlineparams.shape[0]>0):
            run_complete_params = fn_lmbda_split_keyvalue_strings( inlineparams.iloc[0].split(' ') )

            for k,v in run_complete_params.items():
                runseriesitem['run_'+k] = v;

    runbuild.append( runseriesitem );

    # if(dfgrp.shape[0]==4):
    #     break;
dfruns = pd.DataFrame(runbuild);
dfruns.index.set_names(["unit","expname","run"],inplace=True)

# Expeirment With Timeline Plot

In [ ]:
dfruns

In [ ]:
fig = logplotter.mkplot_runs_timeline(dfruns);
fig.show(renderer='browser')

In [ ]:
logs_per_day = dfruns.reset_index().set_index('TimeBeg')['run'].resample('D').count();
[str(x) for x in logs_per_day[logs_per_day==0].index]

# DataExplorer Panel

In [109]:
#dffull = dfbuilt.reset_index().copy();
dffull = dfruns.reset_index().copy();
dffullorig = dffull.copy(); #<-- store a copy so we can come back to this
dfsel = pd.DataFrame();

In [110]:
dffull = dffull[~dffull['TimeBeg'].isna()]

In [111]:
# %% Helpers
def filtered_dataframe(param_values,param_columns):
    
    selection_dict = pd.Series(param_values)[list(param_columns.keys())].to_dict();
    print('selection_dict',selection_dict);

    # this line will filter the dataframe using param_values and param_columns
    #dffiltered = dffull.isin(pd.Series(obj.param.values())[param_columns.keys()].to_dict())[param_columns].all(axis=1);
    #dffiltered = dffull[dffull.isin(pd.Series(param_values)[param_columns.keys()].to_dict())[param_columns].all(axis=1)]

    dffiltered = dffull[dffull.isin(selection_dict)[param_columns.keys()].all(axis=1)]
    
    return dffiltered;

In [112]:
from panel.viewable import Viewer
import param

data_url = "https://assets.holoviz.org/panel/tutorials/turbines.csv.gz"
#turbines = pn.cache(pd.read_csv)(data_url)

c_def_cols = ["unit","expname","run","TimeBeg","status","text"];
c_filters = {
    "expname":param.ListSelector,
    "unit":param.ListSelector,
    "status":param.ListSelector,
    "TimeBeg":param.DateRange
};
# c_def_cols = ["unit","expname","run","Cycle","TimeBeg","status","text"];
# c_filters = {
#     "expname":param.ListSelector,
#     "unit":param.ListSelector,
#     "status":param.ListSelector,
#     "Cycle":param.ListSelector,
#     "TimeBeg":param.DateRange
# };

def update_or_add_parameters(obj,param_columns):
    for col,paramobj in param_columns.items():
        items = dffull[col].unique().tolist();
        
        if(paramobj == param.ListSelector):
            if col not in obj.param:
                # add parameter
                newParam = paramobj(default=sorted(items),objects=sorted(items));
                print('Adding param',col,'dffull.shape',dffull.shape)
                obj.param.add_parameter(col,newParam);
            else:
                # update parameter
                print('Updating param',col,'dffull.shape',dffull.shape)
                # reset available options
                obj.param[col].objects = items;
                # reset selections (to all)
                setattr(obj,col,items);
        elif(paramobj == param.DateRange):
            print('DateRange')
            if col not in obj.param:
                # add parameter
                beg = sorted(dffull['TimeBeg'].unique())[0];
                end = sorted(dffull['TimeBeg'].unique(),reverse=True)[0];
                newParam = paramobj(default=(beg,end),bounds=(beg,end));
                print('Adding param',col,'dffull.shape',dffull.shape)
                obj.param.add_parameter(col,newParam);

class DataExplorer(Viewer):
    data = param.DataFrame(doc="Stores a DataFrame to explore")

    columns = param.ListSelector(
        #default=["p_name", "t_state", "t_county", "p_year", "t_manu", "p_cap"]
        default=c_def_cols,
    )

    filtered_table_data = param.DataFrame(doc="Stores the filtered DataFrame For Table Display")
    filtered_timeline_data = param.DataFrame(doc="Stores the filtered DataFrame For Timeline Display")
    import plotly.graph_objects as go
    timeline_panel = pn.pane.Plotly(
                None,
                sizing_mode='stretch_both', width_policy='max'
            );
    
    tabulator_widget = None;

    def __init__(self, **params):
        super().__init__(**params)
        
        # set parameter for columns to those from dataframe
        self.param.columns.objects = self.data.columns.to_list()

        # dynamically add parameters
        update_or_add_parameters(self,c_filters);

    @param.depends("data", "columns", watch=True, on_init=True)
    def _update_filtered_data(self):
        df = self.data
        # self.filtered_data=df[df.p_year.between(*self.year) & df.p_cap.between(*self.capacity)][
        #     self.columns
        # ]

        #minimum_timeline_columns = ['unit','expname','run','status','TimeBeg','TimeEnd']
        #cols_timeline = list(set(self.columns).union(set(minimum_timeline_columns)));
        cols_table = self.columns;
    
        # TABLE
        # after filtering
        dftable = df[ cols_table ];
    
        # some formatting
        dftable['runraw'] = dftable['run'];
        def mkrunlink(x):
            if((x['status'] == 'run_success') or (x['status'].find('run_ended_early')>=0)):
                #return '<a href="file:///c:\\TEMP\\NAATOS_PM_RUN_runtime_{:s}_{:s}_{:s}.html" target="_blank">{:s}</a>'.format( x['expname'],x['unit'],x['run'],x['run'] );
                return '<a href="rundetail?expname={:s}&unit={:s}&run={:s}" target="_blank">{:s}</a>'.format( x['expname'],x['unit'],x['run'],x['run'] );
            else:
                return x['run'];
            #return 'test';
        ret = dftable.apply( lambda x: mkrunlink(x) ,axis=1);
        dftable['run'] = ret.tolist();

        self.filtered_table_data = dftable;
    

        # TABLE
        # after filtering
        dftimeline = df;
    
        self.filtered_timeline_data = dftimeline;


    @param.depends('filtered_table_data')
    def number_of_rows(self):
        return f"Rows: {len(self.filtered_table_data)}"
    
    def _timelinePlotlyWidget(self):
        print('_timelinePlotlyWidget called');
        #print(self.filtered_timeline_data);
        try:
            fig = logplotter.mkplot_runs_timeline(self.filtered_timeline_data);
            fig.layout.autosize = True;
            self.timeline_panel.object = fig;
        except Exception as e:
            print(e);
            return 'error occurred making timeline'
        return self.timeline_panel;

    @pn.depends(timeline_panel.param.click_data, watch=True)
    def _timeline_click_handling(click_data):
        print("click datatype:{:s} data:{:s}", type(click_data), str(click_data));
        if( type(click_data) == dict ):
            click_expname = click_data['points'][0]['customdata'][0];
            click_run = click_data['points'][0]['customdata'][1];
            click_unit = click_data['points'][0]['y'];
            # # click_expname = click_data.points[0]['customdata'][0];
            # # click_run = click_data.points[0]['customdata'][0];
            # # click_unit = click_data.points[0]['y'];
            print('You clicked on exp:{:s} unit:{:s} run:{:s}'.format(click_expname,click_unit,click_run))

    def _tabulatorWidget(self):
        print('_tabulatorWidget called');

        header_filters = {
            'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
        }
        tabulator_editors = {
            'unit': {'type': 'list', 'valuesLookup': True},
            'status': {'type': 'list', 'valuesLookup': True},
            'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
        }
        pn_summarytable = pn.widgets.Tabulator(
            value=self.param.filtered_table_data,

            pagination='local', page_size=30,
            #pagination='local',page_size=1000,

            #hierarchical=True,
            #groupby=['expname','unit'],
            #theme='midnight',
            #aggregators={"origin": "mean", "yr": "mean"},
            
            show_index=False,
            formatters = {
                'run': dict(type='html'),
                'text': dict(type='textarea'),
            },

            #header_filters=header_filters,
            editors=tabulator_editors,header_filters=True,

            configuration=dict(
                paginationCounter="rows"
            ),
            
            sizing_mode='stretch_both',
        );

        # apply dataframe styling to the tabulator widget
        # https://discourse.holoviz.org/t/dynamic-update-of-tabulator-style/2741
        # STYLING
        def style_statustext(val):
            """
            Takes a scalar and returns a string with
            the css property
            """
            if(val=='run_success'):
                return "background-color: lightgreen";
            elif(val=='run_ended_early_user'):
                return "background-color: lightyellow";
            elif(val=='notrun'):
                return '';
            elif(val.find('boot')>=0):
                return "background-color: powderblue";
            else:
                return "background-color: orchid";
        mydf_styled = pn_summarytable.style.map(style_statustext,subset=pd.IndexSlice[:, ['status']]);


        return pn_summarytable;

    def __panel__(self):
        print('__panel__() method called')
        c_display = set(obj.param.values().keys())-set(('columns','data','filtered_data','filtered_table_data','filtered_timeline_data','name'));
        
        self.tabulator_widget = self._tabulatorWidget();

        return pn.Column(
            pn.Row(
                pn.Column(
                    pn.widgets.MultiChoice.from_param(self.param.columns, width=400),
                ),
                #pn.Column(self.param.year, self.param.capacity),
                *[obj.param[x] for x in c_display]
            ),
            self.number_of_rows,
            
            #pn.widgets.Tabulator(self.param.filtered_data, page_size=10, pagination="remote"),
            
            pn.Tabs(
                ('Timeline',self._timelinePlotlyWidget()),
                ('Table',self.tabulator_widget),
                dynamic=True
            )
        )
#s = DataExplorer(data=turbines).servable()
obj = DataExplorer(data=dffull)
#s = obj.servable();
#obj.servable();


Adding param expname dffull.shape (187, 14)
Adding param unit dffull.shape (187, 14)
Adding param status dffull.shape (187, 14)
DateRange
Adding param TimeBeg dffull.shape (187, 14)


C:\Users\SimonGhionea\AppData\Local\Temp\ipykernel_45556\2711795649.py:93: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SimonGhionea\AppData\Local\Temp\ipykernel_45556\2711795649.py:102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Plotting Panel

In [113]:
def_expname,def_unit,def_run = list( dffull[dffull['status']=='run_success'].groupby(['expname','unit','run']).groups.keys() )[0];
class Plotter1App(Viewer):
    dffull = param.DataFrame(doc="Stores the master dataframe")
    dfbuilt = param.DataFrame(doc="Stores the processed cycle run")

    expname = param.String();
    unit = param.String();
    run = param.String();

    def __init__(self,**params):
        # get first info and set as default property
        print('INIT',def_expname,def_unit,def_run);
        self.param.expname.default = def_expname;
        self.param.unit.default = def_unit;
        self.param.run.default = def_run;
    
        super().__init__(**params);
    
        #pn.state.location.sync(self,['unit'])
    
    def __panel__(self):
        # return pn.Column(
        #     pn.Row(
        #         pn.Column(
        #             pn.widgets.MultiChoice.from_param(self.param.columns, width=400),
        #         ),
        #         #pn.Column(self.param.year, self.param.capacity),
        #         pn.
        #     ),
            
        #     #pn.widgets.Tabulator(self.param.filtered_data, page_size=10, pagination="remote"),
        #     self._tabulatorWidget()

        # pn.Column(

        # )
        # print('2 __panel__');
        # print('2 __panel__',pn.state.location);
        # print(dir(pn.state.location))
        # print(pn.state.session_args);
        
        
        # print(pn.state.location.query_params);
        
        # # # set from query params if they were passed-in
        # # for k,v in pn.state.location.query_params.items():
        # #     if(k in self.param):
        # #         #setattr(self,k,v);
        # #         #self.param.set_param(k,v);
        # #         pass;

        run=self.run;
        unit=self.unit;
        expname=self.expname;
        print('PANEL',expname,unit,run);

        # #pn.state.location.sync(self,['unit'])
        dfplot = self.dffull.query('run==@run and expname==@expname and unit==@unit')
        plotly_fig = logplotter.mkplot_devicerun_detail(dfplot,self.dfbuilt);
        plotly_fig.layout.autosize = True;
        pn_content = pn.pane.Plotly(
            plotly_fig
            ,sizing_mode='stretch_both', width_policy='max'
        )

        # get logfile info
        logfile,configfiles = logreader.getFilesByExpUnitRun(rootpath,experiment=expname,unit=unit,run=run);

        return pn.Tabs(
            # pn.Row(
            #     self.param.expname,self.param.unit,self.param.run
            # ),
            #'PLOT GOES HERE '+str(pn.state.location.query_params),
            #'PLOT GOES HERE '+unit+' '+expname+' '+run,
            ('Run Detail Plot',pn_content),
            ('Logfile',pn.pane.Str('Dumping {:s}\n\n'.format(str(logfile[0]))+logfile[1])),
        );
#obj2 = Plotter1App(dffull = dfraw, dfbuilt = dfbuilt);
#obj2.servable()


# Panel Multi-Page Setup 2

In [115]:
def session_key_func(request):
    key = request.arguments.get('expname', [def_expname.encode()])[0]+'_'+request.arguments.get('unit', [def_unit.encode()])[0]+'_'+request.arguments.get('run', [def_run.encode()])[0];
    print('session_key_func',key);
    return key;

#pn.extension(template='material', session_key_func=session_key_func)
pn.extension(session_key_func=session_key_func)

obj = DataExplorer(data=dffull)
obj2 = Plotter1App(dffull = dfraw, dfbuilt = dfbuilt);

def page1():
    #return obj.servable(location=True)
    return obj;

def page_rundetail():
    #pn.state.location.sync(obj2,['unit']);
    pn.state.location.sync(obj2, ['expname','unit','run']);
    #return obj2.servable()
    return obj2;

ROUTES = {
    "": page1,
    "rundetail": page_rundetail
}
# # if pn.state.location:
# #     pn.state.location.sync(obj2)
try:
    serve.stop();
except Exception as e:
    print(e)
serve = pn.serve(ROUTES, port=5006,location=True,verbose=True,admin=True);

Updating param expname dffull.shape (187, 14)
Updating param unit dffull.shape (187, 14)
Updating param status dffull.shape (187, 14)
DateRange
INIT 20250404_PM_fromDX Unit 16 sample_03-25-25_102548
Launching server at http://localhost:5006


C:\Users\SimonGhionea\AppData\Local\Temp\ipykernel_45556\2711795649.py:93: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SimonGhionea\AppData\Local\Temp\ipykernel_45556\2711795649.py:102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



__panel__() method called
_tabulatorWidget called
_timelinePlotlyWidget called
PANEL 20250404_PM_fromDX Unit 16 sample_04-03-25_114131


c:\Users\SimonGhionea\OneDrive - Global Health Labs, Inc\ProjectsCloud\NAATOS\sgpyanalysisnaatos\naatos_module_tools\logplotter.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



PANEL 20250404_PM_fromDX Unit 22 sample_04-01-25_102155


c:\Users\SimonGhionea\OneDrive - Global Health Labs, Inc\ProjectsCloud\NAATOS\sgpyanalysisnaatos\naatos_module_tools\logplotter.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



PANEL 20250404_PM_fromDX Unit 22 sample_04-01-25_140246


c:\Users\SimonGhionea\OneDrive - Global Health Labs, Inc\ProjectsCloud\NAATOS\sgpyanalysisnaatos\naatos_module_tools\logplotter.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
expname='20250404_PM_fromDX'; unit='Unit 2'; run='sample_04-01-25_134548';
#obj.filtered_table_data.query('expname==@expname and unit==@unit and run==@run')
dftmp = obj.filtered_table_data.query('expname==@expname and unit==@unit and runraw==@run')
print(dftmp[['expname','unit','runraw']])
#obj.

In [ ]:
dir(obj)

In [ ]:
dir(obj.tabulator_widget)

In [ ]:
obj.tabulator_widget.selection=[32]

In [ ]:
tmp = pn.widgets.Tabulator();
tmp.table.

In [ ]:
serve.stop()

In [ ]:
dir(serve)

In [ ]:
dir(serve.get_sessions()[0])

In [51]:
expname = "20250311_DX_download"
unit = "Unit 17"
run = "sample_03-06-25_130017"
dfplot = obj2.dffull.query('run==@run and expname==@expname and unit==@unit')
# #mkplot(dfplot);
plotly_fig = logplotter.mkplot_devicerun_detail(dfplot,dfbuilt);
#plotly_fig.show(render='browser')

logfile,configfiles = logreader.getFilesByExpUnitRun(rootpath,experiment=expname,unit=unit,run=run);

c:\Users\SimonGhionea\OneDrive - Global Health Labs, Inc\ProjectsCloud\NAATOS\sgpyanalysisnaatos\naatos_module_tools\logplotter.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
logfile

(WindowsPath('C:/Users/SimonGhionea/Global Health Labs, Inc/NAATOS Product Feasibility - General - Internal - Electronic Control Module/Beta design/PowermoduleTestData/by_exp/20250311_DX_download/Unit 17/sample_03-06-25_130017.csv'),
 'Time,ValveTemp,ValvePWM,AmpTemp,AmpPWM,Batt,BatteryV,BatteryT,Event\n2025-03-06 13:00:18,0.00,0.00,0.00,0.00,99,4.20,22.55,Cycle 1 Started.\n2025-03-06 13:00:19,21.21,0.00,21.09,0.00,99,4.20,22.39, \n2025-03-06 13:00:20,21.21,0.00,21.09,0.00,99,4.20,22.30, \n2025-03-06 13:00:21,21.20,0.00,21.09,0.00,99,4.20,22.39, \n2025-03-06 13:00:22,21.20,0.00,21.09,0.00,99,4.20,22.43, \n2025-03-06 13:00:23,21.20,0.00,21.09,0.00,99,4.20,22.51, \n2025-03-06 13:00:25,21.20,0.00,21.09,0.00,99,4.20,22.29, \n2025-03-06 13:00:26,21.20,0.00,21.09,0.00,99,4.19,22.51, \n2025-03-06 13:00:27,21.20,0.00,21.09,0.00,99,4.20,22.37, \n2025-03-06 13:00:28,21.20,0.00,21.08,0.00,99,4.20,22.41, \n2025-03-06 13:00:30,21.20,0.00,21.08,0.00,99,4.20,22.41, \n2025-03-06 13:00:31,21.21,0.00,21

In [32]:
dffull.query('run==@run and expname==@expname and unit==@unit')

,unit,expname,run,status,text,CRTCRuntime,TimeBeg,TimeEnd,cycsum_I2CERRCOUNT
52,Unit 17,20250311_DX_download,sample_03-06-25_130017,run_success,CompletedRun @ total runtime 1813s:['Exiting w...,1813.0,2025-03-06 13:00:18,2025-03-06 13:30:31,16.0


In [31]:
logplotter.mkplot_devicerun_detail(dfplot,dfbuilt);

c:\Users\SimonGhionea\OneDrive - Global Health Labs, Inc\ProjectsCloud\NAATOS\sgpyanalysisnaatos\naatos_module_tools\logplotter.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['runtime', 'ValveTemp', 'AmpTemp', 'BatteryT']"

# Manual Plot - Look at RTC time error to tick time error

In [108]:
dftmp = dffull[dffull['status']=='run_success']
print(dftmp.columns)
dftmp = dftmp[['expname','unit','run','run_tick_delta_sec']]
dftmp['run_tick_delta_sec'] = dftmp['run_tick_delta_sec'].astype(float)
fig = px.strip(dftmp,x='unit',y="run_tick_delta_sec");
fig.update_layout(title='Run Timing Mechanism Analysis For {:d} runs with status "run_success" in expnames:{:s}'.format(dftmp.shape[0],str(dftmp['expname'].unique())))
fig.update_yaxes(nticks=20,title='Time-error (SECONDS) between RTC run time and CPUTICK run time')
fig.show(renderer='browser')

Index(['unit', 'expname', 'run', 'status', 'text', 'CRTCRuntime', 'TimeBeg',
       'TimeEnd', 'cycmax_tick_delta_sec', 'cycmax_rtc_delta_sec',
       'cycsum_I2CERRCOUNT', 'run_expected_sec', 'run_tick_delta_sec',
       'run_rtc_delta_sec'],
      dtype='object')


# OTher


In [ ]:
#mydf.to_excel(r'C:\temp\summarytable.xlsx',freeze_panes=(1,1))

In [ ]:
#list(dfraw.groupby(['expname','unit','run']).groups.keys())[0]

In [ ]:
dfbuilt

In [ ]:
dfplot

# Plot tests

In [ ]:
expname = '20250317_SimonGhionea_autodl';
run = 'sample_03-14-25_122320';
unit = 'PM10';
dfplot = dfraw.query('run==@run and expname==@expname and unit==@unit')
# #mkplot(dfplot);
plotly_fig = logplotter.mkplot_devicerun_detail(dfplot,dfbuilt);
#plotly_fig.update_layout(autosize=True);
plotly_fig.show(renderer='browser')

In [ ]:
# timeline
fig = logplotter.mkplot_runs_timeline(dfruns);
fig.update_layout(autosize=False);
#fig.show(renderer='browser',config=dict(response=False))
pn_timeline = pn.pane.Plotly(fig,);
s = pn_timeline.servable();
pn.serve(s)

In [ ]:
fig.show(renderer='browser')

In [ ]:
fig.layout

In [ ]:
logreader.getFilesByExpUnitRun(rootpath)